In [7]:
import pandas as pd
pd_data = pd.read_excel('./data.xlsx',sheet_name="Sheet1")

In [23]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
data = pd_data.values
data = data[:,1:]
sexual = data[:,0]
academy = data[:,1]
encoder = LabelBinarizer()
encoded_sexual = encoder.fit_transform(sexual)
encoded_academy = encoder.fit_transform(academy)
data = np.c_[encoded_sexual,encoded_academy,data[:,2:]]

In [24]:
data

array([[1, 0, 0, ..., 66, 26, 3],
       [1, 0, 0, ..., 45, 14, 3],
       [1, 0, 0, ..., 69, 28, 3],
       ...,
       [1, 0, 0, ..., 120, 40, 0],
       [1, 0, 0, ..., 120, 40, 0],
       [1, 0, 0, ..., 120, 40, 0]], dtype=object)

In [25]:
data.shape

(12828, 39)

In [34]:
features = data[:,:-1]
target_label = data[:,-1].astype(np.int32)

In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)
features_train,features_test,label_train,label_test = train_test_split(features,target_label,test_size = 0.2)
tree_clf = DecisionTreeClassifier(max_depth = 9)
tree_clf.fit(features_train,label_train)
label_predict = tree_clf.predict(features_test)

print(accuracy_score(label_test,label_predict))
accuracy_score(label_train,tree_clf.predict(features_train))

0.8639906469212782


0.9039173650360554

In [101]:
from  sklearn.model_selection import GridSearchCV
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    'min_samples_split':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,116,17,18,19,20] },
  ]
tree_clf = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_clf, param_grid, cv=5,
                           scoring='accuracy', return_train_score=True)
grid_search.fit(features_train, label_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 116, 17, 18, 19, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [102]:
grid_search.best_params_

{'max_depth': 1, 'min_samples_split': 2}

In [112]:
#tree_clf = DecisionTreeClassifier(max_depth = 1)
#tree_clf.fit(features_train,label_train)
label_predict = grid_search.best_estimator_.predict(features_test)

print(accuracy_score(label_test,label_predict))
#accuracy_score(label_train,tree_clf.predict(features_train))
label_predict = grid_search.best_estimator_.predict(features_train)
print(accuracy_score(label_train,label_predict))

0.8846453624318005
0.877996491911908


In [109]:
from sklearn.ensemble import RandomForestClassifier
from  sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=10),
        'max_depth': randint(low=1, high=10),
    }
tree_clf = RandomForestClassifier()
rnd_search = RandomizedSearchCV(tree_clf, param_distributions=param_distribs,
                                # 迭代10次，5折交叉验证
                                n_iter=10, cv=5, scoring='accuracy', random_state=42)
rnd_search.fit(features_train, label_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108f3df98>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108f3de80>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108f3d898>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [111]:
label_predict = rnd_search.best_estimator_.predict(features_test)
print(accuracy_score(label_test,label_predict))
label_predict = rnd_search.best_estimator_.predict(features_train)
print(accuracy_score(label_train,label_predict))

0.8846453624318005
0.8854024556616644


In [113]:
rnd_search.best_params_

{'max_depth': 8, 'max_features': 8, 'n_estimators': 131}